# $Z_3$ model for baryon physics

In [1]:
import numpy as np
import scipy as sp
omega = np.exp(2j*np.pi/3)
V = np.array([[1,0,0],[0,omega,0],[0,0,omega**2]])
U = np.array([[0,1,0],[0,0,1],[1,0,0]])

%load_ext autoreload
%autoreload 2


In [2]:
# Building dense and sparse Kronecker-construction functions for placing a 3x3 operator V on a chosen site (or sites)
# in a chain of N sites (local dimension 3). 
# The operator space dimension is 3**N.
# Dense version uses numpy.kron, sparse version uses scipy.sparse.kron.
# Site indexing: 0-based (0 .. N-1). You can pass a single int or a list/tuple of ints for `sites`.
# The functions validate V shape and the sites list.
# Example usage demonstrated at the end.

import numpy as np

def kron_dense(factors):
    """Helper: compute Kronecker product of a list of dense arrays (left-to-right)."""
    from functools import reduce
    return reduce(np.kron, factors)

def op_on_sites_dense(V, N, sites):
    """
    Construct dense operator on (C^3)^{\otimes N} with operator V placed on the given `sites`.
    Parameters
    ----------
    V : array-like, shape (3,3)
        Local operator to place.
    N : int
        Number of sites (N >= 1).
    sites : int or sequence of ints
        Site index or list of site indices (0-based) where V is placed.
    Returns
    -------
    O : ndarray, shape (3**N, 3**N)
        The dense operator.
    """
    if V.shape != (3,3):
        raise ValueError("V must be a 3x3 matrix.")
    if N < 1 or int(N) != N:
        raise ValueError("N must be a positive integer.")
    if isinstance(sites, int):
        sites = [sites]
    sites = sorted(int(s) for s in sites)
    if any(s < 0 or s >= N for s in sites):
        raise IndexError("Site indices must be in range 0..N-1.")
    
    I3 = np.eye(3, dtype=V.dtype)
    factors = [(V if i in sites else I3) for i in range(N)]
    return kron_dense(factors)


# Sparse version
def op_on_sites_sparse(V, N, sites, format='csr'):
    """
    Construct sparse operator on (C^3)^{\otimes N} with operator V placed on the given `sites`.
    Returns a scipy.sparse matrix (by default CSR).
    """
    import scipy.sparse as sp

    V = sp.csr_matrix(V) if not sp.issparse(V) else V.tocsr()
    if V.shape != (3,3):
        raise ValueError("V must be a 3x3 matrix or sparse 3x3.")
    if N < 1 or int(N) != N:
        raise ValueError("N must be a positive integer.")
    if isinstance(sites, int):
        sites = [sites]
    sites = sorted(int(s) for s in sites)
    if any(s < 0 or s >= N for s in sites):
        raise IndexError("Site indices must be in range 0..N-1.")

    I3 = sp.eye(3, format='csr', dtype=V.dtype)
    result = None
    # left-to-right Kronecker (site 0 is the leftmost factor)
    for i in range(N):
        factor = V if i in sites else I3
        if result is None:
            result = factor
        else:
            result = sp.kron(result, factor, format='csr')
    if format != 'csr':
        result = result.asformat(format)
    return result

In [3]:
from qs_mps.lattice import Lattice
from scipy.sparse import identity, csr_array

class H_Z3_gauss:
    def __init__(
        self,
        Lx,
        Ly,
        model: str,
        lamb: float = 0,
        J: float = 1,
        G: float = 1e3,
    ):
        self.Lx = Lx
        self.Ly = Ly
        self.model = model
        self.charges = np.ones((Ly, Lx+1))
        self.lamb = lamb
        self.J = J
        self.G = G
        self.latt = Lattice((self.Lx+1, self.Ly+1), (False, True))
        self.dof = self.latt.nlinks
        self.sector = self._define_sector()

    def add_charges(self, rows: list, columns: list, vals: list):
        """
        add_charges

        This function adds the charges to the background
        vacuum sector (all positive charges). The number of
        charges we add are given by the len of each indices list

        rows: list - row indices of the charges to add
        columns: list - column indices of the charges to add
        """
        for i, j, q in zip(rows, columns, vals):
            self.charges[j, i] = q

        # self.charges = np.flip(self.charges, axis=0)
        return self

    def _define_sector(self):
        particles = 0
        for charge in self.charges.flatten():
            if charge != 1:
                particles += 1

        if particles == 0:
            sector = "vacuum_sector"
        else:
            sector = f"{particles}_particle(s)_sector"
            self.sector = sector
        return sector

    def local_term(self, link):
        sigma_x = op_on_sites_sparse(V, sites=link, N=self.latt.nlinks)
        return sigma_x

    def plaquette_term(self, loop):
        plaq = op_on_sites_sparse(U, sites=loop[0], N=self.latt.nlinks) @ op_on_sites_sparse(U, sites=loop[1], N=self.latt.nlinks) @ op_on_sites_sparse(U.conjugate(), sites=loop[2], N=self.latt.nlinks) @ op_on_sites_sparse(U.conjugate(), sites=loop[3], N=self.latt.nlinks)
        return plaq

    def gauge_constraint(self, site):
        links = self.latt.star(site=site, L=self.Lx, l=self.Ly)
        G = identity(n=3**self.latt.nlinks)
        filtered_links = [element for element in links if element != 0]
        # print("links:")
        # print(filtered_links)
        for link in filtered_links:
            G = G @ op_on_sites_sparse(V, sites=link - 1, N=self.latt.nlinks)

        return G

    def hamiltonian(self):
        loc = csr_array((3**self.latt.nlinks, 3**self.latt.nlinks))
        # local terms
        for link in range(self.latt.nlinks):
            loc += self.local_term(link)

        plaq = csr_array((3**self.latt.nlinks, 3**self.latt.nlinks))
        # plaquette terms
        for loop in self.latt.plaquettes(from_zero=True):
            plaq += self.plaquette_term(loop)

        # gauge constraint
        Gauss = 0
        I = identity(n=3**self.latt.nlinks)
        for site in self.latt.sites:
            # print(site)
            g = self.gauge_constraint(site)
            Gauss += (g - self.charges[site[1], site[0]] * I) @ (
                g - self.charges[site[1], site[0]] * I
            )
        return -(self.lamb * loc) - (1 / self.lamb * plaq) + (self.G * Gauss)

    def diagonalize(
        self,
        v0: np.ndarray = None,
        sparse: bool = True,
        save: bool = False,
        path: str = None,
        precision: int = 2,
        spectrum: str = "gs",
        cx: list = None,
        cy: list = None,
    ):
        H = self.hamiltonian()

        if sparse:
            if spectrum == "all":
                k = 2 ** len(self.latt.plaquettes())
            elif spectrum == "gs":
                k = 1
            e, v = sp.sparse.linalg.eigsh(H, k=k, which="SA", v0=v0)
        else:
            H = H.toarray()
            e, v = np.linalg.eigh(H)
        if save:
            # print(self.sector)
            np.save(
                path
                + f"/results/eigenvectors/ground_state_direct_lattice_{self.l-1}x{self.L-1}_{self.sector}_{cx}-{cy}_U_{self.G}_hG{self.lamb:.{precision}f}.npy",
                v[:, 0],
            )
        return e, v

In [4]:
Lx = 2
Ly = 2
Z3_baryon = H_Z3_gauss(Lx, Ly, model="Z3", lamb=1)
print(Z3_baryon.charges, Z3_baryon.latt.sites)
print(Z3_baryon.latt._lattice_drawer.draw_lattice())

[[1. 1. 1.]
 [1. 1. 1.]] [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
      13     14     15
      |      |      |      
      +-- 5--+-- 6--+      
      |      |      |      
      10     11     12
      |      |      |      
      +-- 3--+-- 4--+      
      |      |      |      
      7      8      9 
      |      |      |      
      +-- 1--+-- 2--+      
      |      |      |      
      13     14     15



In [5]:
for site in Z3_baryon.latt.sites:
    links = Z3_baryon.latt.star(site=site, L=Lx, l=Ly)
    filtered_links = [element for element in links if element != 0]
    print(links, filtered_links)

[1, 7, 0, 0] [1, 7]
[3, 10, 0, 7] [3, 10, 7]
[5, 0, 0, 10] [5, 10]
[0, 8, 1, 0] [8, 1]
[0, 11, 3, 8] [11, 3, 8]
[0, 0, 5, 11] [5, 11]
[0, 9, 2, 0] [9, 2]
[0, 12, 4, 9] [12, 4, 9]
[0, 0, 6, 12] [6, 12]
